# Python für die Bearbeitung von XML Dokumenten

 
Das XML-format habt ihr im gleichnamigen Modul schon kennengelernt. Dieses Notebook zeigt euch, wie ihr python verwenden könnt um XML-Dokumente zu durchsuchen.

In Teil 1 dieses Notebooks lernt ihr wie man XML-Dokumente in Python einliest und durchsucht, in Teil 2 schauen wir an, wie man die Ergebnisse exportieren kann. Beide Teile arbeiten mit der Beispieldatei `cichlids.xml`, welches zwei Einträge für Buntbarsche enthält. Ihr kennt dieses Beispiel ja bereits aus dem XML-Modul. Teil 3 ist der Startpunkt für euer Miniprojekt. Dort findet ihr 2 Mögliche beispiele, mit Daten und einer kleinen Einführung, die ihr gerne so übernehmen könnt. Falls ihr eigene Ideen habt was ihr machen wollt, könnt ihr das natürlich auch gerne tun.

Ich hoffe ihr findet etwas, das euch interessiert. Bei Fragen meldet euch bei mir unter *nikolaj.bauer@uzh.ch*



## Teil 1: Bearbeiten von XML-Dokumenten mit Python

Damit wir von python aus XML-Dokumente lesen und bearbeiten können gibt es Application Programming Interfaces (APIs). Einer dieser APIs heisst ***lxml***. lxml ist ein Package. Das heisst es ist eine Sammlung von code, die andere Leute geschrieben haben und netterweise der ganzen Welt zur verfügung gestellt haben. Da es nicht zur `Standard-library` von Python gehört müssen wir es installieren:

In [ ]:
%pip install -U -q lxml

Falls oben *Requirement already sasisfied [...]* steht, könnt ihr fortfahren, auch wenn es heisst "you may need to restart the kernel". Falls es etwas installiert hat, müsst ihr das Kernel neu starten. Ihr tut das indem ihr in der Zeile oben auf Restart klickt.

Als nächstes importieren wir eine Reihe von Funktionen von dem Package und lesen ein XML-Dokument ein.

In [ ]:
from lxml import etree  # importiere das modul etree vom package lxml

tree = etree.parse("cichlids.xml")  # einlesen (=parsing) des Dokuments
tree  # Objekt

Wie der Name schon sagt arbeitet das *eTree* Modul mit eine Baumstruktur, die ihr ja bereits aus dem XML Teil kennt. Passenderweise nennen wir das objekt `tree`. Danach verwenden wir  `getroot()` um die Wurzel des Baumes (also das äusserste Element) zu finden.

In [ ]:
root = tree.getroot()
root.tag  # gib den Tag-name des root-elements aus

Da wir das äusserste Element nun als Variable `root` gespeichert haben, können wir durch die Child-Elemente iterieren indem wir eine For-Schleife verwenden: 

In [ ]:
for child in root:  # für jedes child element des Root-Elements:
    print(child.tag, child.attrib)  # drucke den Tag und die Attribute aus

Jedoch müssen wir dank dem lxml-Modul nicht selber durch den Baum iterieren, sondern können direkt per [XPath](https://www.w3schools.com/xml/xpath_intro.asp) nach Elementen suchen: 


In [ ]:
results = root.xpath('//breeding-type')  # suche nach allen elementen, die "breeding-type" heissen
print(results)

Welchen Datentyp hat das Ergebnis? Überlegt es euch kurz, die Antwort kommt in der nächsten Zelle.

In [ ]:
print("type des Resultats: ", type(results))
print("Anzahl Treffer der XPath suche: ", len(results))

Bis jetzt können wir mit dem Resultat noch nicht, viel anfangen, denn die Elemente der Liste sind komplexe Python Objekte, namens `Element`. Dieses Objekt ist definiert im lxml Package. Es enhält nützliche Attribute wie `.text` für den Textinhalt, `.attrib` für die Attribute und `.tag` für den Tag-Namen des Elements. Wir können auf die einzelnen Elemente der Liste mit einem index zugreifen und anschliessend den Text ausdrucken:

In [ ]:
breeding_type_1 = results[0]
breeding_type_2 = results[1]

print(breeding_type_1.text)
print(breeding_type_2.text)

Alternativ können wir mit einer For-Schleife durch die Ergebnisse iterieren. Dies wird nützlich wenn wir viele Ergebnisse haben

In [ ]:
for result in results:
    print(result.text)

***Verständnisfrage:*** Verändere den Code in der oberen Zelle so, dass es anstatt den text, den Namen des Elements ausgibt.

### Aufgabe 1:
Teste deine X-Path-Kenntnisse.
<ol>
    <li>gib alle Farben der Buntbarsche aus (also den Text-content des entsprechenden `color` elementes)</li>
    <li>gib alle IDs der cichlid elemente aus</li>
    <li>suche nach einem Fisch mit dem Namen Zeus</li>
    <li>suche nach einem Fisch mit dem Namen Jupiter (dieser existiert nicht in dem Dokument), was hat das Ergebniss für eine Struktur (welcher Type?)</li>
</ol>

In [ ]:
### Aufgabe 1.1

### Teil 2: Durchsuchen von XML Dateien und Export der Ergebnisse

Wie ihr aus der obigen aufgabe sicherlich bemerkt habt, bekommen wir als Resultat immer eine Liste. Wenn die ***XPath*** expression keine ergebnisse liefert bekommen wir eine leere liste. Dies können wir uns zu Nutze machen, wenn wir suchen wollen ob bestimmte Elemente existieren oder nicht:

In [ ]:
names_to_search = ["Hera", "Zeus", "Jupiter", "Juno"]

for name in names_to_search:  # gehe durch alle Namen, die wir oben definiert haben
    
    # suche nach einem Element mit diesem Namen
    results = root.xpath(f"//name[text()='{name}']")  
    
    if len(results) > 0:  # Wenn wir etwas gefunden haben:
        print(name, "found")  
    else:
        print(name, "not found")


***Exkurs: f-strings*** </br>
Oft wollen wir in einem String Werte von Variablen einsetzen. Dies geht in python besonders einfach mit den sogenannten `f-strings`. Wenn wir einen String mit einem `f` deklarieren (wie das oben in der XPath expression der Fall ist), können wir variablen, die wir im Code definiert haben dort einfügen indem wir sie in `{`geschwungene Klammern`}` setzen.

***Ausgabe der Suche in eine Tabelle*** <br/>
Natürlich wollen wir am Ende ein schönes Resultat haben, das wir präsentieren können. Dafür können wir das Ergebniss in eine `csv`-datei (`C`ommma `S`eparated `V`alues) schreiben, die von einem Programm wie Excel gelesen werden kann. Seht euch den folgenden Code an. Ihr müsst nicht jedes Detail verstehen, aber genug durchblicken, dass ihr den Code für eure Zwecke anpassen könnt.

In [ ]:
names_to_search = ["Hera", "Zeus", "Jupiter", "Juno"]

# öffne eine Datei namens "cichlids_search.csv", im modus "w" (für write)
# falls die Datei nicht existiert wird sie hier von python kreiert. 
# falls sie bereits existiert wird sie überschrieben!!!
with open("cichlids_name_search.csv", "w", encoding="utf-8") as outfile:

    # den header (Namen der Spalten) schreiben
    outfile.write("name,found/not found")  
    outfile.write("\n")  # neue Zeile

    for name in names_to_search:  # gehe durch alle Namen, die wir oben definiert haben
        
        # suche nach einem Element mit diesem Namen
        results = root.xpath(f"//name[text()='{name}']")  
        
        if len(results) > 0:  # Wenn wir etwas gefunden haben:
            outfile.write(f"{name},yes")  
        else:
            outfile.write(f"{name},no")
        outfile.write("\n")

### Aufgabe 2:
Kopiere den Code von der oberen Zelle und verändere ihn so dass:
<ol>
    <li>Es eine datei cichlids_color_search.csv kreiert, die anzeigt welche der Farben `rot, gold, orange und blau' vorkommen.</li>
    <li>Es eine datei cichlids_text.csv kreiert, die eine Tabelle von allen Elementen den Namen (tag) und den Textinhalt anzeigt</li>
</ol>

In [ ]:
### Aufgabe 2.1

In [ ]:
###Aufgabe 2.2

## Teil 3: Eigene Anwendung

Ihr solltet jetzt mit den Basics des durchsuchens von XML-Dateien anhand von Python vertraut sein. Es folgen zwei Anwendungsbeispiele, die ihr euch gerne so als Miniprojekt stellen könnt. Es ist euch dabei überlassen die Aufgaben abzuändern, falls ihr andere Informationen heraus bekommen wollt. Oder falls ihr noch andere Daten in Form von XML aus eurem beruflichen Alltag zur verfügung habt, könnt ihr diese nehmen und eurer eigenen Fragestellung nachgehen. Bei Fragen zögert nicht euch zu melden!

Teil 3a\) beschäftigt sich mit dem automatischen durchsuchen von Webseiten, teil 3b\) mit dem Durchsuchen des Briefnachlasses von Vincent van Gogh.

#### Teil 3a\): Durchsuchen von Internetseiten

Webseiten sind in HTML geschrieben, was auch eine Form von XML ist. Es gibt ein paar festgelegte Elemente, die das Layout definieren, die wichtigsten sind: `<h1>, <h2>, <h3>` für "headings", `<p>` für "paragraph" und `<span>` und `<div>` für komplexere Layouts. 

Das Herunterladen einer Website wird in Python mit dem Modul `requests` gemacht. Dieses muss wie lxml installiert werden.

In [ ]:
%pip install -U -q requests  # installieren des requests package (Kernel neu starten, wenn es etwas installiert hat)


Der unten stehende Code greift auf die homepage von der SRF zu. Ausserdem speichert er das Dokument unter `website.html`, damit ihr euch die Datei in VS-Code ansehen könnt. Ihr müsst den Code nicht vollständig verstehen, aber solltet in der Lage sein ihn anzupassen. Im Folgenden könnt ihr wie weiter oben schon mit der variable `root` arbeiten.

In [ ]:
import requests
from lxml import etree

# Den Inhalt der Webseite Runterladen
response = requests.get("https://www.srf.ch/")
html = response.content

# den xml-element-tree und das root-Element aus dem HTML herauslesen
tree = etree.HTML(html).getroottree()
root = tree.getroot()  # das root element kann jetzt behandelt werden wie bei der xml datei oben

# Die Datei abspeichern, damit wir sie ansehen können :)
tree.write("website.html", pretty_print=True)


Jetzt seid ihr dran: Überlegt euch mit welcher XPath-Expression ihr nur die Text-Elemente der Webseite bekommt. Diesen könnt ihr dann nach Stichworten durchsuchen. Dann sucht euch eine interessante Fragestellung. Zur Zeit wo ich dies schreibe gibt es zum beispiel viele Artikel, die die Credit-Suisse erwähnen. Interessant wäre es vorallem ein solches Script an mehreren Tagen laufen zu lassen und zu sehen was sich verändert. Alternativ könnte man auch verschiedene Zeitungen abfragen. Eurer Kreativität sei hier freier Raum gegeben.

***Tipp 1***: Um genauere Resultate zu erlangen bedenkt, dass es verschiedene Schreibweisen für die selben Dinge gibt (zum Beispiel Credit Suisse, CS, Credit-Suisse, etc.)

***Tipp 2***: Eine Webseite enthält auch andere Felder, die nicht Text für die User(innen), sondern Metainformationen, oder Code (in JavaScript) enthalten. Um solche ergebnisse zu vermeiden, versucht nur die Text-Elemente zu finden, die "children" von den oben genannten Mark-up Elementen (`<h1>`, `<div>`, `<span>`, etc) sind.

### Teil 3b\) : 
Die folgende Aufgabe ist etwas schwieriger als die vorherige, da ihr etwas mehr zusätzliche Informationen braucht.

Es geht um die Korrespondnz von Van Gogh. Im Ordner `vangoghxml` findet ihr 902 Briefe, die im Format der Text Encoding Initiative (TEI), was ebenfalls ein XML-Schema ist, aufbereitet worden sind. [^1] 
Das gibt uns die Möglichkeit Quantitative Fragen wie "von Wem hat Vincent van Gogh die meisten Briefe bekommen?" oder "von welchem Jahr ist die meiste Korrespondenz?" zu beantworten.

Zum anschauen der Files könnt ihr VSCode benutzen, eine noch übersichtlichere Ansicht habt ihr wahrscheinlich, wenn ihr die Dateien mit einem Browser wie Firefox öffnet. Falls ihr etwas nachschauen wollt gibt es die Guidelines der TEI [hier](https://www.tei-c.org/release/doc/tei-p5-doc/en/html/index.html). Es gibt zusätzlich einen eigenen Namespace, den die Autoren verwendet haben, die Dokumentation dazu ist jedoch leider nicht mehr verfügbar (soweit ich weiss). Das klingt alles nach sehr viel auf einmal, aber wenn wir und Schritt für Schritt an die Dateien machen wird sich zeigen was wir tun können.

[^1]: Leo Jansen, Hans Luijten, Nienke Bakker (eds.) (2009), 
'Vincent van Gogh - The Letters'. Amsterdam & The Hague: Van Gogh Museum & Huygens ING. 
http://vangoghletters.org


#### Schritt 1: Lesen von einem Brief
In einem ersten Schritt werden wir einen Brief einlesen und mit XPath durchsuchen. Hier geht es darum das Format kennenzulernen. Im nächsten Schritt werden wir durch alle Briefe iterieren. 
<br/><br/>
Die Autoren haben einen speziellen Namespace benutzt, den wir auch dem lxml Package mitteilen müssen. Danach können wir wie gewohnt XPath verwenden um Elemente zu suchen

In [ ]:
from lxml import etree
tree = etree.parse("vangoghxml/let001.xml")  # lese den ersten Brief ein
root = tree.getroot()


ns_map = {'vg' : "http://www.vangoghletters.org/ns/"} # erstelle Namespace Dictionary (map)
results = root.xpath("//vg:dateLet", namespaces=ns_map)  # Xpath-suche mit dem namespace argument

print(results[0].text)

Die Daten, die uns am meisten interessieren könnten liegen in dem Element `<sourceDesc>`. Dort findet sich allerlei, wie das Datum, der/die Autor(in), etc. Das Bild unten Zeigt diesen Teil des ersten Briefes. Versucht die Code Zelle oben zu verändern um die Daten, die euch interessieren herauszulesen, dann fahrt mit Schritt 2 fort.

![screenshot](Capture.JPG)

#### Schritt 2:


Der Code in der nächsten Zelle erstellt eine Liste mit allen Filenames der Briefe. Ihr könnt diese in einer For-Schleife benutzen um nacheinander auf jeden einzulesen und eure XPath suche zu machen. Stellt sicher, dass ihr das Ergebniss in einer Variable oder einer Datei abspeichert.

In [ ]:
import glob
file_names = glob.glob("vangoghxml/*.xml")  # alle xml dateien im ordner vangoghxml einlesen
file_names

Jetzt solltet ihr alle Tools haben, die ihr braucht um quantitative Fragen über die Korrespondenz von Vincent van Gogh mithilfe von Pyton zu beantworten. Viel Spass !!